In [2]:
import csv
import cv2
import numpy as np
import random
import os

import torchvision
import torch
import torch.nn as nn
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

In [3]:
from google.colab import drive
drive.mount('/content/drive')
TRAIN_PATH = "drive/MyDrive/captcha-hacker/train"
TEST_PATH = "drive/MyDrive/captcha-hacker/test"
device = "cuda"

Mounted at /content/drive


In [5]:

class Task_Dataset(Dataset):
    def __init__(self, data, root, task, return_filename=False):
        if task == 1:
          self.data = [sample for sample in data if sample[0].startswith("task1")]
        if task == 2:
          self.data = [sample for sample in data if sample[0].startswith("task2")]
        if task == 3:
          self.data = [sample for sample in data if sample[0].startswith("task3")]
        self.return_filename = return_filename
        self.root = root
        self.task = task
    
    def __getitem__(self, index):
        
        filename, label = self.data[index]
        img_origin = read_image(f"{self.root}/{filename}")
        img_trans = transform(img_origin)
        if self.return_filename:
            return torch.FloatTensor(img_trans / 255), filename
        else:
          if self.task == 1:
            return torch.FloatTensor(img_trans / 255), int(label)
          elif self.task == 3:
            four_hot_table = [0 for i in range(144)]
            for i in range(4):
              if label[i].isdigit():
                four_hot_table[int(ord(label[i])) - 48 + i * 36] =1
              else:
                four_hot_table[int(ord(label[i])) - 87 + i * 36] =1
            return torch.FloatTensor(img_trans / 255), torch.tensor(four_hot_table)


    def __len__(self):
        return len(self.data)
    
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(), 
    torchvision.transforms.Resize(300), 
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
])

In [13]:
test_data = []
with open(f'drive/MyDrive/captcha-hacker/sample_submission.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        test_data.append(row)
test_ds1 = Task_Dataset(test_data, root=TEST_PATH, task=1, return_filename=True)
test_dl1 = DataLoader(test_ds1, batch_size=50, num_workers=2, drop_last=False, shuffle=False)

##################### file path of the submission.csv   ###############
file = open('drive/MyDrive/submission.csv', 'w', newline='')
csv_writer = csv.writer(file)
csv_writer.writerow(["filename", "label"])

model_task_1 = torchvision.models.resnet18(pretrained=True).to(device)
model_task_1.fc = nn.Linear(in_features=512, out_features=10, bias=True).to(device)

##################### file path of the model   ###############
model_task_1.load_state_dict(torch.load("drive/MyDrive/model1"))
model_task_1.eval()
count = 0
for image, filename in test_dl1:
  image = image.to(device)
  pred = model_task_1(image)
  ans = ""

  # print(pred)
  pred = torch.argmax(pred, dim = 1)
  # print(pred)
  for i in range(50):
    partial_ans = pred[i]
    if partial_ans < 10:
      partial_ans = str(int(partial_ans))
    else:
      partial_ans = chr(partial_ans + 87)
    csv_writer.writerow([filename[i], partial_ans])
  del image, filename

tensor([1, 0, 6, 1, 5, 0, 2, 4, 9, 3, 3, 7, 9, 4, 3, 7, 6, 7, 1, 7, 4, 8, 9, 7,
        2, 4, 3, 2, 9, 6, 5, 2, 7, 2, 6, 4, 2, 7, 6, 5, 7, 2, 9, 6, 9, 1, 2, 1,
        4, 0], device='cuda:0')
tensor([1, 5, 1, 7, 6, 9, 9, 9, 3, 6, 8, 0, 3, 9, 0, 7, 1, 6, 8, 7, 4, 0, 1, 7,
        5, 0, 6, 3, 8, 1, 4, 5, 3, 0, 2, 6, 2, 7, 4, 5, 1, 7, 9, 1, 8, 2, 5, 9,
        6, 9], device='cuda:0')
tensor([7, 0, 3, 7, 7, 1, 6, 3, 6, 4, 1, 5, 8, 9, 2, 0, 7, 7, 3, 2, 6, 2, 9, 6,
        6, 1, 4, 1, 9, 5, 4, 8, 2, 8, 5, 5, 8, 7, 1, 7, 5, 0, 2, 0, 1, 6, 5, 3,
        5, 7], device='cuda:0')
tensor([1, 3, 8, 2, 8, 8, 3, 0, 2, 1, 9, 3, 9, 0, 2, 3, 8, 9, 4, 1, 7, 2, 1, 7,
        6, 9, 7, 8, 3, 8, 4, 4, 0, 9, 4, 1, 9, 6, 5, 5, 7, 1, 3, 2, 4, 8, 7, 8,
        8, 0], device='cuda:0')
tensor([2, 6, 9, 8, 6, 3, 3, 7, 1, 4, 8, 2, 1, 0, 5, 6, 3, 8, 3, 8, 2, 0, 2, 0,
        6, 4, 6, 3, 3, 1, 6, 7, 2, 0, 1, 9, 7, 2, 1, 1, 7, 2, 7, 3, 8, 5, 0, 0,
        6, 5], device='cuda:0')
tensor([9, 1, 5, 6, 6, 7, 0, 7, 0, 8, 1,

KeyboardInterrupt: ignored

In [15]:
test_data = []
with open(f'drive/MyDrive/captcha-hacker/sample_submission.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        test_data.append(row)
test_ds2 = Task_Dataset(test_data, root=TEST_PATH, task=2, return_filename=True)
test_dl2 = DataLoader(test_ds2, batch_size=1, num_workers=2, drop_last=False, shuffle=False)





model_task_2 = torchvision.models.resnet18(pretrained=True).to(device)
model_task_2.fc = nn.Linear(in_features=512, out_features=72, bias=True).to(device)
model_task_2.load_state_dict(torch.load("drive/MyDrive/model2"))
model_task_2.eval()

for image, filename in test_dl2:
  image = image.to(device)
  pred = model_task_2(image)
  ans = ""

  pred = pred.view(-1, 36)
  pred = torch.argmax(pred, dim = 1)
  # print(pred)
  for i in range(2):
    partial_ans = pred[i]
    if partial_ans < 10:
      partial_ans = str(int(partial_ans))
    else:
      partial_ans = chr(partial_ans + 87)
    ans += partial_ans
  # print(ans)
  csv_writer.writerow([filename[0], ans])
  del image, filename

file.close()

r6
jr
vx
mr
0k
28
0n
vr
w9
y3
mf
t5
54
ws
ff
y9
82
hd
kv
dd
ev
s1
d5
93
85
8t
93
vh
y0
1n
v2
89
pk
0y
tn
cy
8e
as
mc
d5
6p
yr
1p
0w
7n
04
6y
s5
h4
42
e9
9y
p1
8p
tp
cp
pa
ak
7m
jx
xf
wn
w3
p9
9t
60
4x
4a
e5
ey
9x
ar
92
xe
a4
2p
ep
ah
ty
32
r0
2s
6a
06
ea
5r
5t
y0
h8
80
87
x6
rh
ph
tk
kp
sr
xp
t3
sc
xx
e3
06
48
n6
hm
er
0m
2f
ss
ek
06
j7
nw
20
sw
mj
67
he
5f
m3
p5
nm
f2
tf
p1
15
k6
d0
4v
pt
3y
4c
px
h4
xy
ea
k2
y1
n8
jp
p5
3f
52
4h
rt
d6
65
n2
wp
6r
c7
d2
8n
3c
te
d6
a9
h7
4j
94
c4
69
y9
r8


KeyboardInterrupt: ignored

In [23]:
test_data = []
with open(f'drive/MyDrive/captcha-hacker/sample_submission.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        test_data.append(row)
test_ds3 = Task_Dataset(test_data, root=TEST_PATH, task=3, return_filename=True)
test_dl3 = DataLoader(test_ds3, batch_size=1, num_workers=2, drop_last=False, shuffle=False)





model_task_3 = torchvision.models.resnet18(pretrained=True).to(device)
model_task_3.fc = nn.Linear(in_features=512, out_features=144, bias=True).to(device)
model_task_3.load_state_dict(torch.load("drive/MyDrive/model3"))
model_task_3.eval()

for image, filename in test_dl3:
  image = image.to(device)
  pred = model_task_3(image)
  ans = ""

  pred = pred.view(-1, 36)
  pred = torch.argmax(pred, dim = 1)
  # print(pred)
  for i in range(4):
    partial_ans = pred[i]
    if partial_ans < 10:
      partial_ans = str(int(partial_ans))
    else:
      partial_ans = chr(partial_ans + 87)
    ans += partial_ans
  # print(ans)
  csv_writer.writerow([filename[0], ans])
  del image, filename

file.close()